In [1]:
# !pip install tqdm

from tqdm import tqdm
import pandas as pd
import requests
import json

In [22]:
df = pd.read_csv('test_for_eda.csv')
df.head()

,시군구,번지,아파트명,전용면적(㎡),계약년월,층,건축년도,도로명,구,회사채금리,매매가격지수,건설공사비지수,버스정류장수,지하철수,계약년,계약월,연식
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.9700,202307,5,1987,언주로 3,강남구,4.437,93.0,127.33,86.0,2.0,2023,7,36
1,서울특별시 강남구 개포동,651-1,개포더샵트리에,108.2017,202308,10,2021,개포로 311,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,2
2,서울특별시 강남구 개포동,652,개포우성3차,161.0000,202307,15,1984,개포로 307,강남구,4.437,93.0,127.33,86.0,2.0,2023,7,39
3,서울특별시 강남구 개포동,652,개포우성3차,133.4600,202308,14,1984,개포로 307,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,39
4,서울특별시 강남구 개포동,652,개포우성3차,104.4300,202308,6,1984,개포로 307,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,39


In [23]:
df['주소'] = df['시군구'] + ' ' + df['번지']
df.head()

,시군구,번지,아파트명,전용면적(㎡),계약년월,층,건축년도,도로명,구,회사채금리,매매가격지수,건설공사비지수,버스정류장수,지하철수,계약년,계약월,연식,주소
0,서울특별시 강남구 개포동,658-1,개포6차우성,79.9700,202307,5,1987,언주로 3,강남구,4.437,93.0,127.33,86.0,2.0,2023,7,36,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,651-1,개포더샵트리에,108.2017,202308,10,2021,개포로 311,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,2,서울특별시 강남구 개포동 651-1
2,서울특별시 강남구 개포동,652,개포우성3차,161.0000,202307,15,1984,개포로 307,강남구,4.437,93.0,127.33,86.0,2.0,2023,7,39,서울특별시 강남구 개포동 652
3,서울특별시 강남구 개포동,652,개포우성3차,133.4600,202308,14,1984,개포로 307,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,39,서울특별시 강남구 개포동 652
4,서울특별시 강남구 개포동,652,개포우성3차,104.4300,202308,6,1984,개포로 307,강남구,4.500,93.4,127.39,86.0,2.0,2023,8,39,서울특별시 강남구 개포동 652


In [18]:
df['주소'] 

0       서울특별시 강남구 개포동 658-1
1       서울특별시 강남구 개포동 651-1
2         서울특별시 강남구 개포동 652
3         서울특별시 강남구 개포동 652
4         서울특별시 강남구 개포동 652
               ...         
9267      서울특별시 중랑구 신내동 816
9268      서울특별시 중랑구 신내동 816
9269      서울특별시 중랑구 신내동 816
9270      서울특별시 중랑구 신내동 816
9271      서울특별시 중랑구 신내동 816
Name: 주소, Length: 9272, dtype: object

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9272 entries, 0 to 9271
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   시군구      9272 non-null   object 
 1   번지       9272 non-null   object 
 2   아파트명     9272 non-null   object 
 3   전용면적(㎡)  9272 non-null   float64
 4   계약년월     9272 non-null   int64  
 5   층        9272 non-null   int64  
 6   건축년도     9272 non-null   int64  
 7   도로명      9272 non-null   object 
 8   구        9272 non-null   object 
 9   회사채금리    9272 non-null   float64
 10  매매가격지수   9272 non-null   float64
 11  건설공사비지수  9272 non-null   float64
 12  버스정류장수   9272 non-null   float64
 13  지하철수     9272 non-null   float64
 14  계약년      9272 non-null   int64  
 15  계약월      9272 non-null   int64  
 16  연식       9272 non-null   int64  
 17  주소       9272 non-null   object 
dtypes: float64(6), int64(6), object(6)
memory usage: 1.3+ MB


In [31]:

url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"

x_coordi = []
y_coordi = []

for a in tqdm(df['주소'], total=len(df['주소'])):
    try:
        params = {
            "query": a,
            "output": "json",
        }

        headers = {
            "x-ncp-apigw-api-key-id": "id",  # Client ID
            "x-ncp-apigw-api-key": "secret"  # Client Secret
        }

        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status()  # HTTP 에러 코드(4xx, 5xx) 발생 시 예외 처리
        data = response.json()

        if "meta" in data and data["meta"].get("totalCount", 0) > 0:
            x = data['addresses'][0].get('x', None)
            y = data['addresses'][0].get('y', None)
        else:
            x, y = None, None

    
    except Exception as e:
        x, y = None, None
        print(f"주소: {a}, 에러: {e}")
    

    x_coordi.append(x)
    y_coordi.append(y)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 9272/9272 [43:01<00:00,  3.59it/s]


In [32]:
data

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '서울특별시 중랑구 신내역로1길 85 신내우디안1단지',
   'jibunAddress': '서울특별시 중랑구 신내동 816 신내우디안1단지',
   'englishAddress': '85, Sinnaeyeok-ro 1-gil, Jungnang-gu, Seoul, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '서울특별시',
     'shortName': '서울특별시',
     'code': ''},
    {'types': ['SIGUGUN'], 'longName': '중랑구', 'shortName': '중랑구', 'code': ''},
    {'types': ['DONGMYUN'], 'longName': '신내동', 'shortName': '신내동', 'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '신내역로1길',
     'shortName': '신내역로1길',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '85',
     'shortName': '85',
     'code': ''},
    {'types': ['BUILDING_NAME'],
     'longName': '신내우디안1단지',
     'shortName': '신내우디안1단지',
     'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '816',
     'shortName': '816',


In [34]:
x_coordi

['127.0568592',
 '127.0572569',
 '127.0560194',
 '127.0560194',
 '127.0560194',
 '127.0685158',
 '127.0716669',
 '127.0716669',
 '127.0716669',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0644350',
 '127.0381516',
 '127.0289105',
 '127.0289105',
 '127.0584123',
 '127.0609864',
 '127.0609864',
 '127.0609864',
 '127.0609864',
 '127.0609864',
 '127.0609864',
 '127.0609864',
 '127.0654620',
 '127.0654620',
 '127.0540217',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0505624',
 '127.0381249',
 '127.0371243',
 '127.0371243',
 '127.0505537',
 '127.05

In [35]:
y_coordi

['37.4762762',
 '37.4848288',
 '37.4839751',
 '37.4839751',
 '37.4839751',
 '37.4875974',
 '37.4888519',
 '37.4888519',
 '37.4888519',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.4811997',
 '37.5147528',
 '37.5138873',
 '37.5138873',
 '37.4963884',
 '37.5008352',
 '37.5008352',
 '37.5008352',
 '37.5008352',
 '37.5008352',
 '37.5008352',
 '37.5008352',
 '37.5010467',
 '37.5010467',
 '37.5056902',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4935294',
 '37.4879855',
 '37.4882030',
 '37.4882030',
 '37.4950706',
 '37.4950706',
 '37.4950706',
 '37.4786400',
 '37.4786400',
 '37.47864

In [36]:
df["x"] = x_coordi
df["y"] = y_coordi

df.to_csv("xylabel_api_data.csv", index=False, encoding="utf-8-sig")

In [41]:
xydf = pd.read_csv("xylabel_api_data.csv")
xydf.isnull().sum()

시군구        0
번지         0
아파트명       0
전용면적(㎡)    0
계약년월       0
층          0
건축년도       0
도로명        0
구          0
회사채금리      0
매매가격지수     0
건설공사비지수    0
버스정류장수     0
지하철수       0
계약년        0
계약월        0
연식         0
주소         0
x          7
y          7
dtype: int64

In [42]:
df[df.isnull().any(axis=1)]

,시군구,번지,아파트명,전용면적(㎡),계약년월,층,건축년도,도로명,구,회사채금리,매매가격지수,건설공사비지수,버스정류장수,지하철수,계약년,계약월,연식,주소,x,y
1265,서울특별시 강북구 미아동,1353,에스케이북한산시티,114.85,202309,14,2004,솔샘로 174,강북구,4.602,93.9,129.34,187.0,5.0,2023,9,19,서울특별시 강북구 미아동 1353,None,None
1919,서울특별시 관악구 봉천동,1152,관악푸르지오102동,114.69,202308,7,2004,관악로30길 27,관악구,4.500,93.4,127.39,219.0,4.0,2023,8,19,서울특별시 관악구 봉천동 1152,None,None
2634,서울특별시 구로구 항동,가-238,하버라인4단지,59.86,202307,14,2019,항동로 72,구로구,4.437,93.0,127.33,38.0,0.0,2023,7,4,서울특별시 구로구 항동 가-238,None,None
2635,서울특별시 구로구 항동,가-238,하버라인4단지,59.86,202307,5,2019,항동로 72,구로구,4.437,93.0,127.33,38.0,0.0,2023,7,4,서울특별시 구로구 항동 가-238,None,None
2636,서울특별시 구로구 항동,가-238,하버라인4단지,59.86,202308,1,2019,항동로 72,구로구,4.500,93.4,127.39,38.0,0.0,2023,8,4,서울특별시 구로구 항동 가-238,None,None
5645,서울특별시 서초구 신원동,690,서초포레스타5단지,59.93,202307,8,2014,청계산로7길 43,서초구,4.437,93.0,127.33,24.0,1.0,2023,7,9,서울특별시 서초구 신원동 690,None,None
9166,서울특별시 중랑구 상봉동,785,건영2차,57.30,202309,18,1996,신내로7길 20,중랑구,4.602,93.9,129.34,32.0,3.0,2023,9,27,서울특별시 중랑구 상봉동 785,None,None
